In [1]:
import pymongo
from pymongo import MongoClient, GEOSPHERE
import shapely.geometry
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element

### 1. Connecting with Foursquare Api.

In [2]:
load_dotenv()
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

In [3]:
office_location = {'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}
print (office_location)

{'type': 'Point', 'coordinates': [40.7592189, -73.9783534]}


### 2. Defining basskeetball stadiums' query

In [4]:
url_query = 'https://api.foursquare.com/v2/venues/explore'
travel_transport = "4d4b7105d754a06379d81259"

In [5]:
# By taking information from Foursquare's API, we are looking for basketball stadiums around our office in a radius of 1000m.

parameters = {"client_id" : tok1,
              "client_secret" : tok2,
              "v": "20180323",
              "ll": f"{office_location.get('coordinates')[0]},{office_location.get('coordinates')[1]}",
              "query":f"travel_transport",
              #"radius":1000
             }

In [6]:
resp = requests.get(url= url_query, params = parameters)
data = json.loads(resp.text)

In [7]:
decoding_data = data.get("response")
decoded = decoding_data.get("groups")[0]
transport = decoded.get("items")

In [8]:
map_ = ["venue","name"]
m_latitude = ["venue","location","lat"]
m_longitude = ["venue","location","lng"]

In [9]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [10]:
print(getFromDict(transport[0],map_))

Theater Row - The Acorn


In [11]:
transport_nyc = []
for dic in transport:
    list1 = {}
    list1["name"] = getFromDict(dic,map_)
    list1["latitud"] = getFromDict(dic,m_latitude)
    list1["longitud"] = getFromDict(dic,m_longitude)
    transport_nyc.append(list1)

In [12]:
# Conversion on dataframe that are used on maps construction.
df_transports = pd.DataFrame(transport_nyc)
df_transports.head()

,name,latitud,longitud
0,Theater Row - The Acorn,40.758551,-73.993419
1,Solstice Travel Vacations,40.758305,-73.967880
2,The Travel Inn,40.760124,-73.996154
3,United Bus and Travel,40.713707,-73.991714
4,Double Happiness Travel,40.713707,-73.991714


**9 transports points** close to the office that facilitates travel for Account Managers.

In [13]:
# We convert our original dataframe into a geodataframe and we add a criteria column.

gdf_transports = gpd.GeoDataFrame(df_transports, geometry = gpd.points_from_xy(df_transports.longitud,df_transports.latitud))

gdf_transports = gdf_transports.assign(name = [ "Theater Row - The Acorn",
                           "Solstice Travel Vacations",
                           "The Travel Inn",
                           "United Bus and Travel",
                           "Double Happiness Travel",
                           "K International Transport Co., Inc",
                           "Delgado Travel",
                           "Active Transport ServicesInc.",
                           "Rivas Travel & Multi Services"],
               criteria = ["transports"]*9)
gdf_transports

,name,latitud,longitud,geometry,criteria
0,Theater Row - The Acorn,40.758551,-73.993419,POINT (-73.99342 40.75855),transports
1,Solstice Travel Vacations,40.758305,-73.967880,POINT (-73.96788 40.75830),transports
2,The Travel Inn,40.760124,-73.996154,POINT (-73.99615 40.76012),transports
3,United Bus and Travel,40.713707,-73.991714,POINT (-73.99171 40.71371),transports
4,Double Happiness Travel,40.713707,-73.991714,POINT (-73.99171 40.71371),transports
5,"K International Transport Co., Inc",40.708588,-74.011889,POINT (-74.01189 40.70859),transports
6,Delgado Travel,40.732078,-74.066660,POINT (-74.06666 40.73208),transports
7,Active Transport ServicesInc.,40.673784,-73.982472,POINT (-73.98247 40.67378),transports
8,Rivas Travel & Multi Services,40.852265,-73.934917,POINT (-73.93492 40.85227),transports


In [14]:
# Convert our dataframe on a dict to be able to use it in MongoDB.

In [15]:
gdf_transports["geometry"] = gdf_transports["geometry"].apply(lambda x: shapely.geometry.mapping(x))
gdf_transports

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry,criteria
0,Theater Row - The Acorn,40.758551,-73.993419,"{'type': 'Point', 'coordinates': (-73.99341890...",transports
1,Solstice Travel Vacations,40.758305,-73.967880,"{'type': 'Point', 'coordinates': (-73.96788024...",transports
2,The Travel Inn,40.760124,-73.996154,"{'type': 'Point', 'coordinates': (-73.9961543,...",transports
3,United Bus and Travel,40.713707,-73.991714,"{'type': 'Point', 'coordinates': (-73.99171447...",transports
4,Double Happiness Travel,40.713707,-73.991714,"{'type': 'Point', 'coordinates': (-73.99171447...",transports
5,"K International Transport Co., Inc",40.708588,-74.011889,"{'type': 'Point', 'coordinates': (-74.01188850...",transports
6,Delgado Travel,40.732078,-74.066660,"{'type': 'Point', 'coordinates': (-74.06666, 4...",transports
7,Active Transport ServicesInc.,40.673784,-73.982472,"{'type': 'Point', 'coordinates': (-73.98247150...",transports
8,Rivas Travel & Multi Services,40.852265,-73.934917,"{'type': 'Point', 'coordinates': (-73.934917, ...",transports


In [16]:
type(gdf_transports.loc[0,"geometry"])

dict

### 3. Export the query to Mongo

In [17]:
client = MongoClient() 
db = client.companies
collection = db.transports 
collection.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [18]:
data = gdf_transports.to_dict(orient='records')
data[0]

{'name': 'Theater Row - The Acorn',
 'latitud': 40.7585514688034,
 'longitud': -73.9934189068339,
 'geometry': {'type': 'Point',
  'coordinates': (-73.9934189068339, 40.7585514688034)},
 'criteria': 'transports'}

In [19]:
collection.insert_many(data)

### 4. Determine the nearest travel transports points.

In [20]:
db.transports 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'transports')

In [21]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[40.7592189, -73.9783534]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.transports .find(filt,proyect).limit(1)
transports = list(result)

In [22]:
transports

[{'name': 'Active Transport ServicesInc.',
  'latitud': 40.67378371119131,
  'longitud': -73.98247150733843}]